# Deploy Embedding Models with TEI on Vertex AI

TL; DR BGE, standing for BAAI General Embedding, is a collection of embedding models released by BAAI, which is an English base model for general embedding tasks ranked in the MTEB Leaderboard. Text Embeddings Inference (TEI) is a toolkit developed by Hugging Face for deploying and serving open source text embeddings and sequence classification models; enabling high-performance extraction for the most popular models, including FlagEmbedding, Ember, GTE and E5. And, Google Vertex AI is a Machine Learning (ML) platform that lets you train and deploy ML models and AI applications, and customize large language models (LLMs) for use in your AI-powered applications. In this example, we will show how to deploy any supported embedding model, in this case [`BAAI/bge-large-en-v1.5`](https://huggingface.co/BAAI/bge-large-en-v1.5), from the Hugging Face Hub in Vertex AI using the TEI DLC available in Google Cloud Platform (GCP) in both CPU and GPU instances.

![`BAAI/bge-large-en-v1.5` in the Hugging Face Hub](./assets/deploy-embedding-on-vertex-ai/model-in-hf-hub.png)

## Setup / Configuration

First, we need to install `gcloud` in our local machine, in order to be able to authenticate to Google Cloud, configure the project we want to use, our preferred / default location, etc.

To install `gcloud`, follow the instructions at https://cloud.google.com/sdk/docs/install.

Then, we will also need to install `google-cloud-aiplatform`, required to programatically create the Vertex AI model, register it in their model registry, and then create the endpoint to deploy the model in Vertex AI. To be installed as follows:

In [ ]:
!pip install google-cloud-aiplatform --upgrade --quiet

Before proceeding, for convenience we will set the following environment variables:

In [ ]:
%env PROJECT_ID=your-project-id
%env LOCATION=your-location
%env BUCKET_URI gs://hf-tei-vertex-ai
%env CONTAINER_URI=us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-embeddings-inference-cu122.1-4.ubuntu2204

Then we need to login into our GCP account and set the project ID to the one we want to use for Vertex AI.

In [ ]:
!gcloud auth login
!gcloud config set project $PROJECT_ID

Once we are logged in, we need to ensure that the necessary APIs are enabled in GCP, such as the Vertex AI, Compute Engine and Container Registry related APIs.

In [ ]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable compute.googleapis.com
!gcloud services enable container.googleapis.com
!gcloud services enable containerregistry.googleapis.com
!gcloud services enable containerfilesystem.googleapis.com

## Optional: Create bucket in GCS

Since we will run a Vertex AI job, we need to specify a GCS bucket to use so as to dump the artifacts, logs, etc. generated from the fine-tuning job; this means that the job will write those into a directory that is mounted into a GCS bucket, meaning that the content is synced with the bucket and persisted there, so that we can use it later on.

We can use an existing bucket, so if you already have a bucket available in GCS for storing the model outputs, feel free to skip this step and jump into the next one.

Otherwise, in order to create the bucket we are going to use `gcloud storage buckets create` to create a new bucket in GCS in the specified project and location.

In [ ]:
!gcloud storage buckets create $BUCKET_URI --project $PROJECT_ID --location=$LOCATION --default-storage-class=STANDARD --uniform-bucket-level-access

## Register model in Vertex AI

Once we are logged in into our GCP account and enabled the required services, after installing `google-cloud-aiplatform` Python SDK, we can already initialize it using our previously defined `PROJECT_ID`, `LOCATION` and `BUCKET_URI`.

In [ ]:
import os
from google.cloud import aiplatform

PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION")
BUCKET_URI = os.getenv("BUCKET_URI")

aiplatform.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET_URI,
)

Then we can already proceed to the model "upload", since it will basically consist on registering the model in Vertex AI with an empty bucket linked to it, since we the model will be automatically downloaded in the Hugging Face TEI DLC as the `MODEL_ID` environment variable is provided.

So on, before going into the code, let's review the arguments:

- `display_name` is the name that will be shown in Vertex AI Model Registry.

- `serving_container_image_uri` is the location of the Hugging Face TEI DLC that we will be using for serving the model later on. In order to see which TEI containers are available in GCP, you can run the following command:

    `gcloud container images list --repository="us-docker.pkg.dev/deeplearning-platform-release/gcr.io" | grep "huggingface-text-embeddings-inference"`

- `serving_container_environment_variables` are the environment variables that will be used during the container runtime, so these are aligned with the environment variables defined by TEI, which in this case natively supports the `AIP_` Vertex AI environment variables (to read more about the environment variables exposed by Vertex AI, please check https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements).
    - `MODEL_ID` is the identifier of the model in the Hugging Face Hub, to check all the TEI supported models please check https://huggingface.co/models?other=text-embeddings-inference&sort=trending.

- `serving_container_ports` this is optional, since we're setting this value to Vertex AI default port which is 8080, but is recommended to have more visibility on the open ports later on.

For more information on the supported `aiplatform.Model.upload` arguments, check the `upload` reference at https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_upload.

In [ ]:
model = aiplatform.Model.upload(
    display_name="BAAI--bge-large-en-v1.5",
    serving_container_image_uri=os.getenv("CONTAINER_URI"),
    serving_container_environment_variables={
        "MODEL_ID": "BAAI/bge-large-en-v1.5",
    },
    serving_container_ports=[8080],
)
model.wait()

![Model in Vertex AI Model Registry](./assets/deploy-embedding-on-vertex-ai/vertex-ai-model.png)

## Deploy model in Vertex AI

Once the model has been registered in Vertex AI, we can define the endpoint we want to deploy the model to, and then link the model deployment to that endpoint resource.

To do so, we'll start by calling `aiplatform.Endpoint.create` to create a new Vertex AI endpoint resource (which comes only with the configuration, it's not linked to a model or anythign usable yet).

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="BAAI--bge-large-en-v1.5-endpoint")

![Vertex AI Endpoint created](./assets/deploy-embedding-on-vertex-ai/vertex-ai-endpoint.png)

Then we can already proceed to the model deployment in an endpoint via the `deploy` method within the previously registered `model`. The `deploy` method will link the previously created endpoint resource with the model that contains the configuration of the serving container, TEI in this case, and then it will deploy that model in Vertex AI in the specified instance/s.

So on, before going into the code, let's review the arguments:

- `endpoint` is the endpoint to deploy the model to, which is optional and by default will be set to the model's display name plus `_endpoint`; but in this case we're using a previously created endpoint.
- `machine_type`, `accelerator_type` and `accelerator_count` are arguments that define which instance to use, and additionally, if desired, also the accelerator to use (GPU or TPU) and the number of accelerators, respectively. The `machine_type` and the `accelerator_type` are tied together, since when using an instance with an accelerator, we will need to select an instance that supports it, to read more about the different instances check https://cloud.google.com/compute/docs/gpus, and to read about the `accelerator_type` naming check https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec.
- `sync` is an optional argument on whether to deploy the model and wait until it's done i.e. sync, or just trigger the deployment and continue the code execution i.e. async. In this case we set it to True (default value), as we won't be able to succesfully run the follow up cells until the endpoint is deployed.

For more information on the supported `aiplatform.Model.deploy` arguments, check the `deploy` reference at https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_deploy.

In [ ]:
deployed_model = model.deploy(
    endpoint=endpoint,
    machine_type="g2-standard-4",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
    sync=True,
)

**WARNING**: _The Vertex AI endpoint deployment via the `deploy` method may take from 15 to 25 minutes._

![Vertex AI Endpoint running the model](./assets/deploy-embedding-on-vertex-ai/vertex-ai-endpoint-run.png)

![Vertex AI Endpoint logs in Cloud Logging](./assets/deploy-embedding-on-vertex-ai/vertex-ai-endpoint-logs.png)

## Run online inference in Vertex AI

Finally, we can run the online predictions on Vertex AI using the `predict` method, which will basically send the requests to the running endpoint in predict route specified within the container.

In [ ]:
output = deployed_model.predict(instances=[{"inputs": "What is Deep Learning?"}])
output.predictions[0][0][:5], len(output.predictions[0][0])

Which produces the following output (truncated for brevity, but original tensor length is 1024, which is the embedding dimension of `BAAI/bge-large-en-v1.5` i.e. the model we're serving):

```
([0.018108826, 0.0029993146, -0.04984767, -0.035081815, 0.014210845], 1024)
```

![Vertex AI Endpoint logs in Cloud Logging after predict](./assets/deploy-embedding-on-vertex-ai/vertex-ai-endpoint-logs-predict.png)

## Resource clean-up

Finally, we can release the resources we have created as follows:

- `deployed_model.undeploy_all` to undeploy the model from all the endpoints.
- `deployed_model.delete` to delete the endpoint/s where the model was deployed gracefully after the `undeploy_all`.
- `model.delete` to delete the model from the registry i.e. unregister it. Note that when using a Google Cloud Storage (GCS) artifact, this method won't delete neither the bucket nor its contents, but only unregister the model from Vertex AI.

In [ ]:
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()

Optionally, we can also remove the GCS Bucket we created before, with the following `gcloud` command:

In [ ]:
!gcloud storage rm -r $BUCKET_URI